In [4]:
import pandas as pd

dev_df = pd.read_csv('devset/posts.csv',sep="\t")
test_df = pd.read_csv('testset/posts_groundtruth.csv',sep="\t")

dev_df.rename(columns={'image_id(s)':'image_id'},inplace=True)

dev_df[:10]

,post_id,post_text,user_id,image_id,username,timestamp,label
0,324597532548276224,Don't need feds to solve the #bostonbombing wh...,886672620,"boston_fake_03,boston_fake_35",SantaCruzShred,Wed Apr 17 18:57:37 +0000 2013,fake
1,325145334739267584,PIC: Comparison of #Boston suspect Sunil Tripa...,21992286,boston_fake_23,Oscar_Wang,Fri Apr 19 07:14:23 +0000 2013,fake
2,325152091423248385,I'm not completely convinced that it's this Su...,16428755,boston_fake_34,jamwil,Fri Apr 19 07:41:14 +0000 2013,fake
3,324554646976868352,Brutal lo que se puede conseguir en colaboraci...,303138574,"boston_fake_03,boston_fake_35",rubenson80,Wed Apr 17 16:07:12 +0000 2013,fake
4,324315545572896768,4chan and the bombing. just throwing it out th...,180460772,boston_fake_15,Slimlenny,Wed Apr 17 00:17:06 +0000 2013,fake
5,324581777614180352,4chan thinks they found pictures of the bomber...,46224814,boston_fake_08,iamyadvinder,Wed Apr 17 17:55:00 +0000 2013,fake
6,324665423956176896,"Ola ke ase, investigando las bombas de Boston ...",90735851,boston_fake_35,rcr866,Wed Apr 17 23:27:23 +0000 2013,fake
7,325464125868216321,4chan ThinkTank - Imgur http://t.co/hQt2fhxE48,142785938,"boston_fake_03,boston_fake_35",GlebGgs,Sat Apr 20 04:21:09 +0000 2013,fake
8,325099014355820544,@DLoesch have you seen this? Bomber #2 looks ...,21769179,boston_fake_13,larrygloverlive,Fri Apr 19 04:10:19 +0000 2013,fake
9,325705653026975744,da 4chan think tank BOSTON http://t.co/0ZbKMA...,51410043,"boston_fake_03,boston_fake_35",IsraelTucker,Sat Apr 20 20:20:53 +0000 2013,fake


In [5]:
import shutil
import os
image_count = {}

def move_image():
    source_image_dir1 = 'devset/images'
    source_image_dir2 = 'testset/images'
    
    dev_image_set = {file for file in os.listdir(source_image_dir1) if not file.endswith('.txt')}
    test_image_set = {file for file in os.listdir(source_image_dir2) if not file.endswith('.txt')}
    
    distinct_image_dir = 'images'        
    os.makedirs(distinct_image_dir, exist_ok=True)
    for file in dev_image_set:
        source_file = f'{source_image_dir1}/{file}'
        destination_file = f'{distinct_image_dir}/{file}'
        shutil.copy(source_file, destination_file)
    
    for file in test_image_set:
        source_file = f'{source_image_dir2}/{file}'
        destination_file = f'{distinct_image_dir}/{file}'
        shutil.copy(source_file, destination_file)
    
    image_set = {file.split('.')[0] for file in dev_image_set} | {file.split('.')[0] for file in test_image_set}
    return image_set
    
def get_illegal_image(image_list,image_set):
    if image_list is None:
        return None
    if isinstance(image_list,str):
        image_list = [image_list]
    image_intersection = set(image_list).intersection(image_set)
    if len(image_intersection) == 0:
        return None
    
    result_image = max(image_intersection, key=lambda x: -image_count[x])
    
    for im in image_intersection:
        image_count[im] = image_count[im] + 1
    return result_image

def filter_no_image(df,image_set):
    print(f'before filter : {df.shape[0]}')
    df['image_id'] = df['image_id'].map(lambda x: get_illegal_image(x,image_set))
    df = df[df['image_id'].notna()]
    print(f'after filter : {df.shape[0]}')
    return df
    
    
    
    
    
image_set = move_image()
image_count = { image:0  for image in image_set }
dev_df = filter_no_image(dev_df,image_set)
test_df = filter_no_image(test_df,image_set)

before filter : 15629
after filter : 13097
before filter : 2177
after filter : 1098


In [6]:
df = pd.concat([dev_df,test_df],ignore_index=True)
df.to_csv('twitter.csv',index=False)